## Group 7
## Big Data Group Project

### Contributors:
- 22606127: Sean Whitehead [Sean]

### Phase 2: Cleansing and Reshaping

This notebook is used for co-developing of codes required to wrangle the home credit risk dataset.

Please use this notebook as the live version and download local copies as you require.

If you have picked up a specific section, please comment your name to the code cell so that we can work on other sections.

Helpful link for Dataframes in Scala:
- https://docs.databricks.com/spark/latest/dataframes-datasets/introduction-to-dataframes-scala.html#work-with-dataframes
- https://spark.apache.org/docs/1.5.0/api/java/overview-summary.html
- https://spark.apache.org/docs/2.2.0/sql-programming-guide.html
- https://www.tutorialspoint.com/scala/

Insert libraries into the cell below:

In [10]:
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql._
import org.apache.hadoop.io.LongWritable
import org.apache.hadoop.io.Text
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat
import org.apache.spark.sql.functions.{sum, col}

import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import org.apache.spark.sql._
import org.apache.hadoop.io.LongWritable
import org.apache.hadoop.io.Text
import org.apache.hadoop.conf.Configuration
import org.apache.hadoop.mapreduce.lib.input.TextInputFormat


Insert code for data loading below:

Download zip from GDrive and extract to location where you are running notebook for below to run

In [2]:
//read in csv [Sean]
val creditRiskdf = spark.read.option("header","true").csv("Home credit default risk after feature selection/m_train_small.csv")

creditRiskdf: org.apache.spark.sql.DataFrame = [EXT_SOURCE_1: string, EXT_SOURCE_2: string ... 359 more fields]


In [15]:
//take a look at features in dataset (...361 of them) [Sean]
creditRiskdf.printSchema()

root
 |-- EXT_SOURCE_1: string (nullable = true)
 |-- EXT_SOURCE_2: string (nullable = true)
 |-- EXT_SOURCE_3: string (nullable = true)
 |-- client_installments_AMT_PAYMENT_min_sum: string (nullable = true)
 |-- DAYS_BIRTH: string (nullable = true)
 |-- AMT_CREDIT: string (nullable = true)
 |-- AMT_ANNUITY: string (nullable = true)
 |-- DAYS_EMPLOYED: string (nullable = true)
 |-- bureau_DAYS_CREDIT_ENDDATE_max: string (nullable = true)
 |-- bureau_DAYS_CREDIT_max: string (nullable = true)
 |-- DAYS_ID_PUBLISH: string (nullable = true)
 |-- bureau_AMT_CREDIT_MAX_OVERDUE_mean: string (nullable = true)
 |-- bureau_DAYS_ENDDATE_FACT_max: string (nullable = true)
 |-- bureau_AMT_CREDIT_SUM_DEBT_mean: string (nullable = true)
 |-- OWN_CAR_AGE: string (nullable = true)
 |-- previous_loans_CNT_PAYMENT_mean: string (nullable = true)
 |-- client_cash_CNT_INSTALMENT_FUTURE_min_max: string (nullable = true)
 |-- previous_loans_SELLERPLACE_AREA_max: string (nullable = true)
 |-- DAYS_LAST_PHONE_C

 |-- client_installments_NUM_INSTALMENT_NUMBER_mean_min: string (nullable = true)
 |-- client_bureau_balance_STATUS_C_count_mean: string (nullable = true)
 |-- client_installments_NUM_INSTALMENT_NUMBER_sum_min: string (nullable = true)
 |-- client_credit_CNT_DRAWINGS_CURRENT_max_sum: string (nullable = true)
 |-- client_credit_AMT_PAYMENT_CURRENT_sum_sum: string (nullable = true)
 |-- previous_loans_NAME_PORTFOLIO_XNA_count_norm: string (nullable = true)
 |-- previous_loans_NAME_CLIENT_TYPE_Repeater_count_norm: string (nullable = true)
 |-- previous_loans_NAME_SELLER_INDUSTRY_Connectivity_count_norm: string (nullable = true)
 |-- bureau_CREDIT_TYPE_Credit card_count: string (nullable = true)
 |-- client_credit_AMT_CREDIT_LIMIT_ACTUAL_sum_sum: string (nullable = true)
 |-- previous_loans_NAME_CONTRACT_STATUS_Canceled_count_norm: string (nullable = true)
 |-- client_bureau_balance_STATUS_0_count_norm_max: string (nullable = true)
 |-- OCCUPATION_TYPE_Drivers: string (nullable = true)
 |-

 |-- previous_loans_PRODUCT_COMBINATION_Card Street_count_norm: string (nullable = true)
 |-- client_installments_NUM_INSTALMENT_NUMBER_max_min: string (nullable = true)
 |-- client_credit_AMT_PAYMENT_CURRENT_min_sum: string (nullable = true)
 |-- previous_loans_PRODUCT_COMBINATION_POS industry without interest_count_norm: string (nullable = true)
 |-- previous_loans_CHANNEL_TYPE_Regional / Local_count: string (nullable = true)
 |-- ORGANIZATION_TYPE_Transport: type 3: string (nullable = true)
 |-- client_cash_NAME_CONTRACT_STATUS_Returned to the store_count_norm_max: string (nullable = true)
 |-- ORGANIZATION_TYPE_School: string (nullable = true)
 |-- previous_loans_NAME_TYPE_SUITE_Other_B_count_norm: string (nullable = true)
 |-- bureau_CNT_CREDIT_PROLONG_mean: string (nullable = true)
 |-- client_credit_AMT_DRAWINGS_CURRENT_max_max: string (nullable = true)
 |-- client_bureau_balance_MONTHS_BALANCE_max_sum: string (nullable = true)
 |-- client_cash_NAME_CONTRACT_STATUS_Active_count_

In [7]:
//ID col summary (min to max) & number of rows
creditRiskdf.describe(creditRiskFeatures(360)).show()

2019-05-25 05:31:44,414 WARN  [Thread-4] util.Utils (Logging.scala:logWarning(66)) - Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.debug.maxToStringFields' in SparkEnv.conf.


res5: Long = 307511


In [9]:
//string array of features to pass through EDA functions [Sean]
val creditRiskFeatures = creditRiskdf.columns

creditRiskFeatures: Array[String] = Array(EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_AMT_CREDIT_MAX_OVERDUE_mean, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, OWN_CAR_AGE, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, clie...

## Split and allocate features for data cleaning task execution (MapReduce style?)

In [23]:
//split and allocate features for down stream cleaning [Sean]
val creditRiskFeatures1 = creditRiskFeatures.slice(0,120)
val creditRiskFeatures2 = creditRiskFeatures.slice(121,240)
val creditRiskFeatures3 = creditRiskFeatures.slice(241,360)

creditRiskFeatures1: Array[String] = Array(EXT_SOURCE_1, EXT_SOURCE_2, EXT_SOURCE_3, client_installments_AMT_PAYMENT_min_sum, DAYS_BIRTH, AMT_CREDIT, AMT_ANNUITY, DAYS_EMPLOYED, bureau_DAYS_CREDIT_ENDDATE_max, bureau_DAYS_CREDIT_max, DAYS_ID_PUBLISH, bureau_AMT_CREDIT_MAX_OVERDUE_mean, bureau_DAYS_ENDDATE_FACT_max, bureau_AMT_CREDIT_SUM_DEBT_mean, OWN_CAR_AGE, previous_loans_CNT_PAYMENT_mean, client_cash_CNT_INSTALMENT_FUTURE_min_max, previous_loans_SELLERPLACE_AREA_max, DAYS_LAST_PHONE_CHANGE, bureau_AMT_CREDIT_SUM_max, bureau_AMT_CREDIT_SUM_mean, DAYS_REGISTRATION, client_installments_DAYS_INSTALMENT_max_max, previous_loans_AMT_DOWN_PAYMENT_max, CODE_GENDER_F, REGION_POPULATION_RELATIVE, client_cash_CNT_INSTALMENT_max_mean, bureau_DAYS_CREDIT_UPDATE_max, bureau_AMT_CREDIT_SUM_sum, cli...

In [ ]:
//create partitioned DFs for downstream cleaning [Sean]
//pass sliced array of features as SQL eg from creditRiskdf select slice1 ....

Insert code for flagging missing values below:

In [ ]:
//**NEW APPROACH**
//create DF of features & count of nulls
//set threshold e.g if 20% of values = null, drop column
//**EXTENTION** Impute values to replace null, will have to review feature properties ***IF WE HAVE TIME***

//sum count of int for null values [Lin]
creditRiskdf.select(creditRiskdf.columns.map(c => sum(col(c).isNull.cast("int")).alias(c)): _*).show

Insert code for flagging duplicate values below:

In [ ]:
//because the data has already been transformed & normalised use the IDCol (ARRindex:360) to check dupes

Insert code for visualising or displaying outliers below:

In [ ]:
//write function to check if there are any outliers in a given domain and show boxplot [Sean]
//set approach for defining outliers ie 3s.d away from mean/median
//decide whether to remove or not
//Q1 - 1.5*IQR and Q3 + 1.5*IQR
def testOutliers(feature: String, sDevThresh: int): Array = {
    var testVal = mean + sDevThresh * //std Deviation of feature_label;
    var testDf = //DF(select feature, idCol from df where feature > testVal);
    
    //if (length of testDF > 0 ){
        //return testDf}
}

In [ ]:
//test for outliers [Sean]
var sDevThresh = 3
for (feature <- creditRiskFeatures){
    testOutliers(feature, sDevThresh);
}

In [ ]:
//handle outliers **Do we drop them or keep them?
//

Insert code for reviewing data values for inconsistencies below:

In [18]:
//this is going to be hard with so many features...

Insert code for reviewing data format below:

In [ ]:
//iterate through features and apply the describe method and append to new dataframe
//secondary to this iterate through the values in the features and get datatypes, sql query this output for unique values

In [ ]:
// find out if there are data in date formats [Joel]
// filter column headers containing 'date' for further investigation
val matcher = ".+date.+"
val dates = creditRiskFeatures.filter(x => x.toLowerCase matches matcher)

In [ ]:
// review data format of columnbs with headers containing 'date' [Joel]
for (dateformat <- dates){
creditRiskdf.select(dateformat).show(10)
}

Insert code for encoding categorical variables below:

In [ ]:
//this has already been done for this dataset...
//we can map the approach perhaps? ie Target (default/notdefault = 1/0)